In [1]:
from __future__ import print_function
import numpy
import sys
import os
import configparser
from all_parameters import get_all_parameters
import tensorflow as tf
import argparse
from utils import compute_values, get_data
from experiment import RunExperiment
from workspace import workspace
from model import MatchingNetwork
from vocabulary import get_word_info
import math
import random
import json
import time
from tensorflow.python.client import device_lib
import pickle

/home/raghab-ankan/miniconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/raghab-ankan/miniconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/raghab-ankan/miniconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/raghab-ankan/miniconda3/e

In [2]:
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

def train_test_model(params, sess, model, experiment):
    TEST_FILE_INDEX = 2
    train_data = get_data(params,
                          params['mnet_training_list'],
                          role='train_workspace')
    
    dev_data = get_data(params,
                        params['mnet_dev_list'],
                        role='valid_workspace')

    test_data = get_data(params,
                         params['testing_list'],
                         role='test_workspace')

    best_v_macro_avg_eer = 1.0
    total_epoches = params['total_epoch']
    print('Starting epochs, Total =', total_epoches)

    for e in range(1, total_epoches+1):
        avg_loss = 0.0

        for workspace_idx in numpy.random.permutation(range(len(train_data))):
            curr_train_workspace = train_data[workspace_idx]
            all_ood_workapces = train_data[:workspace_idx] + train_data[workspace_idx+1:]
            batch_loss = experiment.run_training_epoch(curr_train_workspace, all_ood_workapces)
#             if params['single_workspace'] is False or e % 500 == 0:
#             if e%500 == 0:
#                 print("Epoch {}: train_loss: {}".format(e, batch_loss))
#             sys.stdout.flush()
            avg_loss += batch_loss
        avg_loss = avg_loss / len(train_data)
        print('Epoch {:4d}: avg_loss: {} {}'.format(e, avg_loss, os.getpid()))

#         if e % 50 == 0 or total_epoches==100:
        if 1==1:
            sys.stdout.flush()
            print('-'*40, 'Testing', '-'*40)
            v_macro_avg_eer, v_macro_avg_far, v_macro_avg_frr, \
                v_macro_avg_acc_ideal, v_macro_avg_acc, \
                val_output_info_list = compute_values(params, experiment, dev_data, e)

#             t_macro_avg_eer, t_macro_avg_far, t_macro_avg_frr, \
#                 t_macro_avg_acc_ideal, t_macro_avg_acc, \
#                 test_output_info_list = compute_values(params, experiment, test_data, e)

            print("Meta-Valid Macro(eer, onacc_ideal, onacc): %.3f, %.3f, %.3f" %
                  (v_macro_avg_eer, 
                   v_macro_avg_acc_ideal, 
                   v_macro_avg_acc))
#             print("Meta-Test Macro(eer, onacc_ideal, onacc): %.3f, %.3f, %.3f" %
#                   (t_macro_avg_eer,
#                    t_macro_avg_acc_ideal,
#                    t_macro_avg_acc))
            print('-'*89)

In [3]:
config_paths = ["../config/config"]
config_parser = configparser.SafeConfigParser()
config_found = config_parser.read(config_paths)

params = get_all_parameters(config_parser, "test-run")
params['model_string'] = "test-run"

os.environ["CUDA_VISIBLE_DEVICES"] = '1'

tf.set_random_seed(params['seed'])
numpy.random.seed(params['seed'])
random.seed(params['seed'])

print('Parameters:', json.dumps(params, indent=4))
sys.stdout.flush()

# build the vocabulary
# if mode is train, vocabulary is train_data word + old word vectors
# otherwise, vocabulary is read from wordd2idx file, params["wordvectors"]
#            is initialized randomly, restored by model file later
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

#     print(get_available_gpus())
#     time.sleep(5)

with tf.Session(config=config) as sess:
    voc, w2v = get_word_info(params)

    params['vocabulary'] = voc
    voclist = [None] * len(voc)
    for v in voc:
        voclist[voc[v]] = v
    params['voclist'] = voclist
    params["wordvectors"] = w2v

#     with open('params.txt', 'w') as outfile:
#         json.dumps(params, outfile)

#     with open('params.pkl', 'wb') as f:
#         pickle.dump(params, f)

All parameters:
	total_epoch==>200
	single_workspace==>False
	w2vfile==>../glove/glove.6B.100d.txt
	mnet_training_dir==>../data/train/
	mnet_training_list==>../data/train/workspace_list
	mnet_dev_dir==>../data/dev/
	mnet_dev_list==>../data/dev/workspace_list
	testing_dir==>../data/test/
	testing_list==>../data/test/workspace_list
	model_string==>dummy
	model_dir==>/dccstor/slad/mingtan/offtopic_paper/outputs/models
	batch_size==>10
	max_length==>40
	emb_size==>100
	encoding==>utf-8
	min_ss_size==>100
	hidden_size==>200
	softmax_factor==>10.0
	learning_rate==>0.001
	remove_target_from_support_set==>True
	dropout_keep_prob==>1.0
	vocabulary==>None
	wordvectors==>None
	offtopic_label==>UNCONFIDENT_INTENT_FROM_SLAD
	sampling_classes==>2
	topk_ss==>0
	layer_num==>1
	enable_batchnorm==>True
	mtl_num_tasks==>0
	filter_size==>3
	ood_threshold==>0.6
	ood_threshold_margin==>0.2
	ood_example_size==>1
	alpha_pos==>1.0
	alpha_neg==>1.0
	alpha_indomain==>1.0
	seed==>3143
Parameters: {
    "total_epo

/home/raghab-ankan/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The SafeConfigParser class has been renamed to ConfigParser in Python 3.2. This alias will be removed in future versions. Use ConfigParser directly instead.
  


get_word_info, params["w2vfile"] -> ../glove/glove.6B.100d.txt
Embed Size 100
Skipped -> 0
wdmap 400000
len(W) 400000
Loaded BERT Model
0 words taken from BERT
Time Taken = 0.0008 minutes
After combined with train file
word2idx size: 400028


In [4]:
model = MatchingNetwork(params)

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
dir(model)

['W',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'build',
 'classify',
 'dn',
 'get_prototype',
 'get_train_op',
 'init',
 'input_ood_sents',
 'input_support_set_sents',
 'input_target_sent',
 'input_target_sent_test',
 'is_training',
 'ood_sents',
 'ood_sents_mask',
 'params',
 'ss_encoded_sents_avg_test',
 'ss_g',
 'support_set_labels',
 'support_set_sents',
 'support_set_sents_mask',
 'target_label',
 'target_sent',
 'target_sent_mask',
 'target_sent_mask_test',
 'target_sent_test']

In [7]:
model.init()

Instructions for updating:
Use keras.layers.conv2d instead.
batch norm enabled.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

g_conv_encoder Tensor("losses/g/truediv:0", shape=(?, 200), dtype=float32)
batch norm enabled.
g_conv_encoder Tensor("losses/g_1/truediv:0", shape=(?, 200), dtype=float32)
batch norm enabled.
g_conv_encoder Tensor("losses/g_2/truediv:0", shape=(?, 200), dtype=float32)
batch norm enabled.
g_conv_encoder Tensor("losses/g_3/truediv:0", shape=(?, 200), dtype=float32)
current_learning_rate 0.001
tf.global_variables()--> <tf.Variable 'W:0' shape=(400028, 100) dtype=float32_ref>
tf.global_variables()--> <tf.Variable 'g/conv1/kernel:0' shape=(3, 1, 100, 200) dtype=float32_ref>
tf.global_variables()--> <tf.Variable 'g/BatchNorm/beta:0' shape=(200,) dtype=float32_ref>
tf.global_variabl

In [14]:
model.variables

[<tf.Variable 'g/conv1/kernel:0' shape=(3, 1, 100, 200) dtype=float32_ref>,
 <tf.Variable 'g/BatchNorm/beta:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'g/BatchNorm/gamma:0' shape=(200,) dtype=float32_ref>]

In [ ]:
tf.reset_default_graph()
with tf.Graph().as_default():
    sess = tf.Session()
    with sess.as_default():

        model = MatchingNetwork(params)
        model.init()
        experiment = RunExperiment(model, params, sess)
        sess.run(tf.global_variables_initializer())
        train_test_model(params, sess, model, experiment)
        print("=====Training Finished======")